In [ ]:
import pandas as pd
import streamlit as st
from polyfuzz import PolyFuzz


In [ ]:

st.set_page_config(page_title="Medicine utilisation", page_icon=":guardsman:", layout="wide")
st.image("chailogo.png", width=200)
st.title("Facility name matching")

file_1 = st.file_uploader("Upload the first CSV file", type=["csv"])
file_2 = st.file_uploader("Upload the second CSV file (optional)", type=["csv"])
standard_names_file = st.file_uploader("Upload names file", type=["csv", "txt"])

if file_1:
    df1 = pd.read_csv(file_1)
    st.dataframe(df1.head())

    if file_2:
        df2 = pd.read_csv(file_2)
        st.dataframe(df2.head())

        if standard_names_file:
            standard_names_df = pd.read_csv(standard_names_file, header=None)
            fuzzer = PolyFuzz('TF-IDF')
            fuzzer.fit(standard_names_df[0])
            df_appended = pd.concat([df1, df2], axis=0)
            df_appended["standard_name"] = df_appended["description"].apply(lambda x: fuzzer.get_best(x))
            st.dataframe(df_appended.head())
            
            matched_pct = len(df_appended[df_appended["standard_name"].notnull()]) / len(df_appended) * 100
            unmatched_pct = len(df_appended[df_appended["standard_name"].isnull()]) / len(df_appended) * 100
            st.write(f"Percentage of successfully matched facility names: {matched_pct:.2f}%")
            st.write(f"Percentage of unmatched facility names: {unmatched_pct:.2f}%")
            
            if st.button('Download updated facility names as TXT'):
                df_appended["standard_name"].to_csv("updated_facility_names.txt", index=False, header=False)
                st.success('Updated facility names downloaded as TXT')
else:
    st.warning("No file was uploaded.")